In [5]:
# NOTE: THIS HAS ALL PLAYOFFS AND REGULAR SEASON POINTS INTO CONSIDERATION
import os

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from mplbasketball import Court

# Team configuration
# Scarborough_Shooting_Stars
# Ottawa_BlackJacks
# Calgary_Surge
# Edmonton_Stingers
#Montreal_Alliance
TEAM_NAME = "Scarborough_Shooting_Stars"
DISPLAY_NAME = TEAM_NAME.replace("_", " ")
# NOTE ALL IMAGES MUST BE 400x400
BASE_IMG_URL = f"./img/{TEAM_NAME.split('_')[0]}/{{player_name}}.png"
AUTHOR = "By OJ Adeyemi"

# Court visualization settings
COURT_SETTINGS = {
    "orientation": "vu",
    "origin": "bottom-left",
    "court_color": "#2B2B2B",
    "line_color": "#FFFFFE",
    "title_color": "#F0F0F0",
    "title_bg_color": "#0A2A66",
    "subtitle_color": "#CCCCCC",
}

# Visual styling settings
VISUAL_STYLE = {
    "player_box": {
        "facecolor": "black",
        "alpha": 0.8,
        "edgecolor": "#444444",
        "boxstyle": "round,pad=0.3",
        "linewidth": 1,
    },
    "stat_box": {
        "facecolor": "black",
        "alpha": 0.85,
        "edgecolor": "#555555",
        "boxstyle": "round,pad=0.2",
        "linewidth": 0.5,
    },
}

# Statistics settings
MIN_ATTEMPTS = 10

# Zone coordinates on the court
ZONE_COORDINATES = {
    "left_corner": (-48, 90),
    "left_short_corner": (-40, 86),
    "left_wing_three": (-40, 69),
    "restricted_area": (-25, 89),
    "right_corner": (-2, 90),
    "right_short_corner": (-10, 86),
    "right_wing_three": (-10, 69),
    "top_key_midrange": (-25, 72),
    "top_key_three": (-25, 63),
}

# Setup and load data
os.makedirs("visualizations", exist_ok=True)
file_path = f"team_zone_top_scorers/{TEAM_NAME}_zone_top_scorers.csv"
df = pd.read_csv(file_path)


### HELPER FUNCTIONS


In [6]:
import matplotlib.offsetbox as offsetbox
from matplotlib.axes import Axes


def get_player_image(player_name: str):
    """Load and resize a player's image"""
    player_img_path = BASE_IMG_URL.format(player_name=player_name)
    try:
        img = mpimg.imread(player_img_path)
        return OffsetImage(img, zoom=0.18)
    except FileNotFoundError:
        print(f"Image not found: {player_img_path}")
        return None


def setup_court():
    """Create and configure the court figure"""
    court = Court(court_type="nba", origin=COURT_SETTINGS["origin"], units="ft")
    fig, ax = plt.subplots(figsize=(18, 12), facecolor="black")

    # Draw the court
    court.draw(
        ax,
        court_color=COURT_SETTINGS["court_color"],
        line_color=COURT_SETTINGS["line_color"],
        line_width=0.3,
        orientation=COURT_SETTINGS["orientation"],
    )
    return fig, ax


def style_title(ax: Axes, title_text: str):
    """Apply styling to the title"""
    ax.set_title(
        title_text,
        fontsize=20,
        color=COURT_SETTINGS["title_color"],
        fontweight="bold",
        pad=20,
    )

    # Add a background bar for the title
    fig = plt.gcf()
    fig.text(
        0.5,
        0.96,
        " ",
        transform=ax.transAxes,
        ha="center",
        fontsize=24,
    )


def add_player_to_plot(
    ax: Axes, x: float, y: float, player: str, formatted_stat_text: str
):
    """Add a player's image and stats to the plot"""
    player_img = get_player_image(player)
    if player_img:
        # Add player image
        ab = AnnotationBbox(player_img, (x, y), frameon=False)
        ax.add_artist(ab)

        # Add player name with black background
        ax.text(
            x,
            y - 3.2,
            player,
            fontsize=9,
            color="white",
            ha="center",
            fontweight="bold",
            bbox=dict(
                facecolor="black", alpha=0.7, edgecolor="none", boxstyle="round,pad=0.3"
            ),
        )

        # Add stat value with black background and white text
        ax.text(
            x,
            y - 4.2,
            formatted_stat_text,
            fontsize=8,
            color="white",
            ha="center",
            fontweight="bold",
            bbox=dict(
                facecolor="black", alpha=0.7, edgecolor="none", boxstyle="round,pad=0.3"
            ),
        )


def add_text_with_logo(ax: Axes, x, y, text, logo_path="./img/linkedin.png"):
    # Add text with white color and no background
    ax.text(
        x,
        y,
        text,
        color="white",
        style="italic",
        fontweight="semibold",
    )

    # Load LinkedIn logo and place it next to text
    img = mpimg.imread(logo_path)
    imagebox = offsetbox.AnnotationBbox(
        offsetbox.OffsetImage(img, zoom=0.04),
        (x - 0.7, y),
        frameon=False,
        box_alignment=(0.4, 0.2),
    )
    imagebox2 = offsetbox.AnnotationBbox(
        offsetbox.OffsetImage(img, zoom=0.04),
        (x - 0.7, y),
        frameon=False,
        box_alignment=(1.4, 0.2),
    )


### CREATING VIZUALIZATION


In [7]:
SUBTITLE_X, SUBTITLE_Y = -16, 99


# 1. Total Points Visualization
def create_total_points_viz():
    top_scorers = (
        df.sort_values("total_points", ascending=False)
        .groupby("zone")
        .first()
        .reset_index()
    )
    fig, ax = setup_court()

    # Add a subtle gradient background
    ax.patch.set_facecolor("#202020")

    for _, row in top_scorers.iterrows():
        zone, player = row["zone"], row["player"]
        points = int(row["total_points"])

        if zone in ZONE_COORDINATES:
            x, y = ZONE_COORDINATES[zone]
            formatted_text = f"{points} pts"
            add_player_to_plot(ax, x, y, player, formatted_text)

    style_title(ax, f"{DISPLAY_NAME}\nAll-Time Top Scorers (Total Points)")
    #add_text_with_logo(ax, SUBTITLE_X, SUBTITLE_Y, "By OJ Adeyemi")

    plt.savefig(
        f"visualizations/{TEAM_NAME}_top_scorers_total.png",
        dpi=300,
        bbox_inches="tight",
        facecolor="black",
    )
    plt.close()


# 2. Field Goal Percentage Visualization
def create_fg_pct_viz():
    # Only include players with minimum attempts
    qualified_df = df[df["total_attempts"] >= MIN_ATTEMPTS]
    top_fg_scorers = (
        qualified_df.sort_values("fg_percentage", ascending=False)
        .groupby("zone")
        .first()
        .reset_index()
    )
    fig, ax = setup_court()

    # Add a subtle gradient background
    ax.patch.set_facecolor("#202020")

    for _, row in top_fg_scorers.iterrows():
        zone, player = row["zone"], row["player"]
        fg_pct = row["fg_percentage"]
        attempts = int(row["total_attempts"])

        if zone in ZONE_COORDINATES:
            x, y = ZONE_COORDINATES[zone]
            formatted_text = f"{fg_pct * 100:.1f}% FG ({attempts} att)"
            add_player_to_plot(ax, x, y, player, formatted_text)

    style_title(ax, f"{DISPLAY_NAME}\nAll-Time Efficient Scorers (Field Goal %)")
    #add_text_with_logo(ax, SUBTITLE_X, SUBTITLE_Y, "By OJ Adeyemi")

    plt.savefig(
        f"visualizations/{TEAM_NAME}_top_scorers_fg_pct.png",
        dpi=300,
        bbox_inches="tight",
        facecolor="black",
    )
    plt.close()


# Generate visualizations
print(f"Creating visualizations for {DISPLAY_NAME}...")
create_total_points_viz()
create_fg_pct_viz()
print("Completed! Check the 'visualizations' folder.")

Creating visualizations for Scarborough Shooting Stars...
Completed! Check the 'visualizations' folder.
